In [31]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from gerrychain import Graph, GeographicPartition, Election, MarkovChain, tree, constraints
from functools import partial
from gerrychain.updaters import Tally, cut_edges
from gerrychain.metrics import polsby_popper
from gerrychain.proposals import recom
import os
from gerrychain.accept import always_accept

from combination import *
from candidate import Candidate
from ballot import Ballot

In [32]:
PROCESSED_SHAPEFILE_PATH = os.path.join(os.getcwd())
VA_PATH = os.path.join(PROCESSED_SHAPEFILE_PATH, "virginia", "VA_precincts.shp")
EXPORT_PATH = os.path.join(PROCESSED_SHAPEFILE_PATH, "virginia")

ITERATIONS = 1
SAFE_SEAT_THRESHOLD = 0.66
MINORITY_POP_THRESHOLD = RACE_WINNER_THRESHOLD = 0.5
TOTAL_DISTRICTS = 10
FIVE_FIVE = 2
THREE_THREE_FOUR = 3.35


In [33]:
va_graph = Graph.from_file(VA_PATH, ignore_errors=True)

/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)
/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:66: UserWarning: Found overlaps among the given polygons. Indices of overlaps: {(887, 892), (893, 915), (892, 914), (887, 893)}
  warnings.warn(
/Users/alexisperezmendoza/Desktop/python-mggg/mggg/lib/python3.10/site-packages/gerrychain/graph/graph.py:296: UserWarning: NA values found in column G17DGOV!
  warnings.warn("NA values found in column {}!".format(column))


In [34]:
for node in va_graph.nodes:
    # Convert all votes to numbers to avoid NaNs 
    va_graph.nodes[node]["G18DHOR"] = float(va_graph.nodes[node]["G18DHOR"])
    va_graph.nodes[node]["G18RHOR"] = float(va_graph.nodes[node]["G18DSEN"])
    if not va_graph.nodes[node]["G18DHOR"]:
        print(node)
    if not va_graph.nodes[node]["G18RHOR"]:
        print(node)


In [35]:
total_population = sum(va_graph.nodes[node]["TOTPOP"] for node in va_graph.nodes)


In [36]:
partition_334 = tree.recursive_tree_part(va_graph, [1, 2, 3], total_population / THREE_THREE_FOUR, "TOTPOP", 0.01, 1)

va_election = Election("CongressionalRace" , {"Democratic": "G18DHOR", "Republican": "G18RHOR"})

In [37]:
new_partition334 = GeographicPartition(va_graph, assignment=partition_334, updaters={
        "population": Tally("TOTPOP", alias="population"),
        "white": Tally("NH_WHITE", alias="white"),
        "black": Tally("NH_BLACK", alias="black"),
        "asian": Tally("NH_ASIAN", alias="asian"),
        "hisp": Tally("HISP", alias="hisp"),
        "other": Tally("NH_OTHER", alias="other"),
        "cut_edges": cut_edges,
        "CongressionalRace": va_election
    })

In [38]:
ideal_pop55 = total_population / FIVE_FIVE
proposal55 = partial(recom, pop_col="TOTPOP", pop_target=ideal_pop55, epsilon=.01, node_repeats=2)
compactness55 = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(new_partition334["cut_edges"])
)
pop_constraint55 = constraints.within_percent_of_ideal_population(new_partition334, 0.3)


In [39]:
chain334 = MarkovChain(
    proposal=proposal55,
    constraints=[
        compactness55,
        pop_constraint55
    ],
    accept=always_accept,
    initial_state=new_partition334,
    total_steps=ITERATIONS
)


In [98]:
import random 

ballot_map = {}
vote_map = {}
all_candidates_dem = [Candidate("Wexton, Jennifer", "Democratic"), Candidate("Spanberger, Abigail Davis.", "Democratic"),
                     Candidate("Scott, Robert C.", "Democratic"), Candidate("Luria, Elaine G.", "Democratic"), 
                     Candidate("Connolly, Gerald E.", "Democratic"), Candidate("Beyer, Donald S. Jr.", "Democratic"),
                     Candidate("Bridenstine, Jimmy", "Democratic"), Candidate("Carter, James", "Democratic"),
                     Candidate("Smith, Steve", "Democratic"), Candidate("Cramer, David", "Democratic"),
                      ]
                     
all_candidates_rep = [Candidate("Wittman, Robert J", "Republican"), 
                     Candidate("Giffith, H. Morgan", "Republican"), Candidate("Good, Bob", "Republican"), 
                     Candidate("Cline, Ben", "Republican"), Candidate("Hudson, Richard", "Republican"), 
                     Candidate("McHenry, Patrick T.", "Republican"), Candidate("Murphy, Gregory", "Republican"), 
                     Candidate("Rouzer, David", "Republican"),
                     Candidate("Stevens, Jax", "Republican"), Candidate("White, Steve", "Republican"),
                     ]


for step in chain334:
    for part in step.parts:
        ballots = []

        total_reps = round((step["population"][part] / total_population) * TOTAL_DISTRICTS)
        dem_candidates = []
        rep_candidates = []
      
        for i in range(total_reps):
            dem_candidates.append(all_candidates_dem.pop())

        for i in range(total_reps):
            rep_candidates.append(all_candidates_rep.pop())

        rep_votes = 0
        dem_votes = 0
        for node in step.parts[part]:
            random.shuffle(dem_candidates)
            random.shuffle(rep_candidates)
            # Create ballots for the Republican and Democratic candidates
            ballots += [Ballot([rep for rep in rep_candidates]) for _ in range(round(int(va_graph.nodes[node]["G18RHOR"])))]
            ballots += [Ballot([dem for dem in dem_candidates]) for _ in range(round(int(va_graph.nodes[node]["G18DHOR"])))]
            rep_votes += round(int(va_graph.nodes[node]["G18RHOR"]))
            dem_votes += round(int(va_graph.nodes[node]["G18DHOR"]))
        ballot_map[part] = ballots
        vote_map[part] = (dem_votes + rep_votes, total_reps)


In [99]:
from collections import Counter

election_results = {}
initial_votes = {}

for district in ballot_map:
    ballots = ballot_map[district]
    winning_threshold = vote_map[district][0] / (1+vote_map[district][1])
    vote_counts = Counter()

    remaining_candidates = []
    eliminated_candidates = []
    winners = []
    winner_count = 0
    candidates_dict = {}
    rounds = 0

    for ballot in ballots:
        first_candidate = ballot.rankings[0]
        vote_counts[first_candidate] += ballot.weight
    if district == 1 or district == 2:
        remaining_candidates = vote_counts.most_common((vote_map[district][1] * 2 + 2))
    else:   
        remaining_candidates = vote_counts.most_common((vote_map[district][1] * 2))

    remaining_candidates.sort(key=lambda x: x[1], reverse=True)
    initial_votes[district] = remaining_candidates

    while(True):

        remaining_candidates = vote_counts.most_common((vote_map[district][1] * 2) - rounds - winner_count)
        remaining_candidates.sort(key=lambda x: x[1], reverse=True)

        if len(remaining_candidates)-1 + len(winners) == vote_map[district][1]:
            winners += remaining_candidates
            eliminated_candidates.append(winners.pop())
            break

        # SURPLUS ROUND
        first_place = remaining_candidates[0]
        if first_place[1] >= winning_threshold:
            remaining_candidates.remove(first_place)
            surplus = first_place[1] - winning_threshold
            sp_count = 0
            for ballot in ballots:
                if ballot.rankings[0] == first_place[0]:
                    ballot.rankings.remove(ballot.rankings[0])
                    ballot.set_weight(ballot.get_weight() * .8)
                    vote_counts[ballot.rankings[0]] = round(ballot.weight + vote_counts[ballot.rankings[0]], 4)
                    sp_count += 1
                    if sp_count >= surplus:
                        break
            for ballot in ballots:
                if ballot.rankings[0] == first_place[0]:
                    vote_counts[ballot.rankings[0]] -= 1

            first_place = (first_place[0], winning_threshold)
            winners.append(first_place) 
            rounds +=1
            winner_count += 1
            continue

        # ELIMINATION ROUND
        eliminated = remaining_candidates[(vote_map[district][1] * 2) - rounds-1]
        if len(remaining_candidates)-1 + len(winners) == vote_map[district][1]:
            remaining_candidates.remove(eliminated)
            winners += remaining_candidates
            eliminated_candidates.append(eliminated)
            break
        for ballot in ballots:
            if ballot.rankings[0] == eliminated[0]:
                ballot.rankings.remove(ballot.rankings[0])
                ballot.set_weight(ballot.get_weight() * .8)
                vote_counts[ballot.rankings[0]] = round(ballot.weight + vote_counts[ballot.rankings[0]], 4)
                
        for ballot in ballots:
            if ballot.rankings[0] == eliminated[0]:
                vote_counts[ballot.rankings[0]] -= 1

        eliminated_candidates.append(eliminated)
        rounds += 1
        election_results[district] = {"Winners": winners, "Eliminated": eliminated_candidates, "Initial Votes": initial_votes[district]}


In [100]:
# make the candidates json serializable
for district in election_results:
    winners = election_results[district]["Winners"]
    eliminated = election_results[district]["Eliminated"]
    initial_votes = election_results[district]["Initial Votes"]
    election_results[district]["RepCountTotal"] = vote_map[district][1]
    election_results[district]["VoteThreshold"] = vote_map[district][0] / (1+vote_map[district][1])
    election_results[district]["Winners"] = [(winner[0].name, winner[0].party, winner[1]) for winner in winners]
    election_results[district]["Eliminated"] = [(eliminated[0].name, eliminated[0].party, eliminated[1]) for eliminated in eliminated]
    election_results[district]["Initial Votes"] = [(initial_votes[0].name, initial_votes[0].party, initial_votes[1]) for initial_votes in initial_votes]


In [101]:
import json
# Export the results to a JSON file
with open('va_election_results.json', 'w') as fp:
    json.dump(election_results, fp)
    
